In [1]:
# Note: honestly, I was pretty tempted to create a model that preps data based on type of data and model it will feed into,
# but opted to choose this simplified generalized version due to time constraints

In [2]:
# here is the summary of agents used and in order:
# Standardize: Fix types, remove currency symbols, unify text casing.
# Date: Extract features (Year/Month) from timestamps.
# Duplicates: Remove exact row matches (Safety: <1%).
# Grouper: Group rare categories into "Other" (Safety: <1%).
# Nulls: Drop bad cols/rows or Impute (Median/Mode).
# Correlation: Drop redundant features (Correlation > 95%).
# Skew: Log-transform positive outliers.
# Encoding: Convert categories to numbers (One-Hot vs Label).
# Scaler: Split Train/Test and Standardize (Prevent Leakage).
# Bias: Fixes Statistical Bias and Class Imbalance on Train set.
# Mastermind: orchestrates the code of these 10 agents to work in sequential order on a given dataset.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# setup keys
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [5]:
# import libraries
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.code_executors import UnsafeLocalCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
# configure retry options
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [7]:
import pandas as pd
import numpy as np
import random
# using a randomized dataset, in production replace with an actual dataset
# Set seed for reproducibility
np.random.seed(42)

def generate_messy_dataset(rows=1000):
    print("⚠️ Generating The Doomed Dataset...")
    
    # 1. BASE DATA & SKEW (Triggers SkewAgent)
    # Generate a log-normal distribution (Right skewed)
    transaction_amt = np.random.lognormal(mean=2, sigma=1, size=rows)
    
    # 2. REDUNDANT FEATURES (Triggers CorrelationAgent)
    # Celsius and Fahrenheit are perfectly correlated
    temp_c = np.random.normal(25, 5, rows)
    temp_f = temp_c * 9/5 + 32
    
    # 3. MESSY STRINGS & CURRENCY (Triggers StandardizeAgent)
    # Includes whitespace, different cases, and symbols
    cities = ["  new york ", "New York", "SF", "sf ", "chicago", "Chicago", "  Austin"]
    city_col = np.random.choice(cities, rows)
    
    salaries = np.random.randint(40000, 150000, rows).astype(str)
    # Corrupt 30% of salaries with currency symbols
    for i in range(rows):
        if np.random.rand() < 0.3:
            salaries[i] = f"${salaries[i]}"
        if np.random.rand() < 0.1:
            salaries[i] = f"{salaries[i]},00" # European style comma/decimal mix
            
    # 4. DATES (Triggers DateAgent)
    # Mix of formats and NaTs
    start_date = pd.to_datetime('2020-01-01')
    dates = [start_date + pd.Timedelta(days=x) for x in range(rows)]
    date_strings = [d.strftime('%Y-%m-%d') for d in dates]
    # Corrupt some dates
    date_strings[0] = "Not a Date"
    date_strings[10] = "Unknown"
    
    # 5. NULLS & MISSING DATA (Triggers NullAgent)
    # A. > 50% Missing (Should be dropped entirely)
    mostly_empty = np.array([np.nan] * rows)
    mostly_empty[:10] = 1 # Only 10 values exist
    
    # B. < 5% Missing (Rows should be dropped)
    tiny_missing = np.random.rand(rows)
    tiny_missing[:15] = np.nan # 1.5% missing
    
    # C. ~20% Missing (Should be Imputed)
    medium_missing_age = np.random.randint(18, 70, rows).astype(float)
    medium_missing_age[:200] = np.nan # 20% missing
    
    # 6. RARE CATEGORIES (Triggers GrouperAgent)
    # 'Google' and 'Direct' are common; 'Friend' and 'Billboard' are rare (<1%)
    sources = ['Google']*800 + ['Direct']*190 + ['Friend']*5 + ['Billboard']*5
    np.random.shuffle(sources)
    
    # 7. HIGH CARDINALITY (Triggers EncodingAgent - Label Encode)
    # 50 unique ZIP codes
    zips = np.random.randint(90000, 90050, rows).astype(str)
    
    # 8. LOW CARDINALITY (Triggers EncodingAgent - One-Hot Encode)
    membership = np.random.choice(['Gold', 'Silver', 'Bronze'], rows)
    
    # 9. CLASS IMBALANCE (Triggers AutoBalanceAgent)
    # 90% Class 0, 10% Class 1
    target = np.random.choice([0, 1], rows, p=[0.90, 0.10])
    
    # CREATE DATAFRAME
    df = pd.DataFrame({
        'ID_Column': range(rows), # Should be ignored by Skew/Scaling agents
        'Transaction_Amt': transaction_amt, # Skewed
        'Temp_C': temp_c, # Redundant
        'Temp_F': temp_f, # Redundant to be dropped
        'City': city_col, # Messy text
        'Salary': salaries, # Messy numbers ($)
        'Join_Date': date_strings, # Date parsing
        'Garbage_Col': mostly_empty, # >50% null
        'Sensor_Reading': tiny_missing, # <5% null
        'User_Age': medium_missing_age, # Impute median
        'Referral': sources, # Group 'Friend' -> Other
        'Zip_Code': zips, # Label Encode
        'Membership': membership, # One-Hot Encode
        'Target_Label': target # Imbalanced
    })
    
    # 10. DUPLICATES (Triggers DuplicatesAgent)
    # Append top 5 rows to bottom to create exact duplicates (0.5% duplicates)
    df = pd.concat([df, df.head(5)], ignore_index=True)
    
    print("✅ Dataset Created. Shape:", df.shape)
    return df

# Initialize
df = generate_messy_dataset()

# Peek at the mess
print("\n--- The Messy Data Head ---")
print(df.head())
print("\n--- The Class Imbalance ---")
print(df['Target_Label'].value_counts())

# doing this since we need to access it in post review and I dont want it to be in the model,
# since that wouldn't make this "for general use"
# SAFE_PATH = "/kaggle/working/"
df = generate_messy_dataset()
df.to_pickle('/kaggle/working/df.pkl')

# This ensures all agents operate in the same folder and files persist (instead of to google cloud).
local_executor = UnsafeLocalCodeExecutor()

⚠️ Generating The Doomed Dataset...
✅ Dataset Created. Shape: (1005, 14)

--- The Messy Data Head ---
   ID_Column  Transaction_Amt     Temp_C     Temp_F      City     Salary  \
0          0        12.142530  31.996777  89.594199   Chicago      83764   
1          1         6.434896  29.623168  85.321703    Austin      41134   
2          2        14.121360  25.298152  77.536673   Chicago  102943,00   
3          3        33.886946  21.765316  71.177569        SF     135667   
4          4         5.846520  28.491117  83.284010        SF      94318   

    Join_Date  Garbage_Col  Sensor_Reading  User_Age Referral Zip_Code  \
0  Not a Date          1.0             NaN       NaN   Google    90033   
1  2020-01-02          1.0             NaN       NaN   Direct    90036   
2  2020-01-03          1.0             NaN       NaN   Google    90019   
3  2020-01-04          1.0             NaN       NaN   Direct    90007   
4  2020-01-05          1.0             NaN       NaN   Direct    90006 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
# Note: have to output store df, train_df, and test_df safe path
# because the agent framework's execution sandbox does not persist local variable assignments or
# global scope modifications back to the main environment after the agent finishes.

In [9]:
# standardize text based on appearance percentage and similarity, strings to number if possible
standardize_agent = LlmAgent(
    name="StandardizeAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using the `pandas` library to clean and standardize a dataset.

    **CRITICAL RULE:** DO NOT create sample data. You MUST load the existing file. Load 'df.pkl', standardize, overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - Load DataFrame variable named `df`by: df = pd.read_pickle('/kaggle/working/df.pkl')
    - Do NOT write code to load the file (e.g., do not use pd.read_csv).
    - Focus ONLY on the transformation logic.

    **TRANSFORMATION LOGIC:**
    1. **Column Standardization:** Ensure column names are stripped of whitespace and lowercased.
    2. **Type Inference:** Detect columns that look like numbers (e.g., "$1,200", "500") and convert them to numeric types, handling non-numeric characters gracefully.
    3. **Categorical Standardization:** For text columns with low cardinality, strip whitespace and unify casing (e.g., "  ny " -> "NY").
    4. **Date Parsing:** Identify columns containing dates and convert them to datetime objects using `pd.to_datetime` with `errors='coerce'`.

    **OUTPUT RULES:**
    1. Your output MUST be ONLY a Python code block (```python ... ```).
    2. Do NOT write any text before or after the code block.
    3. The code MUST end by:
        1. printing the first 5 rows of the cleaned `df` using `print(df.head())`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Standardized and saved to df.pkl")
    5. Use ONLY standard libraries and `pandas`. Do NOT import `fuzzywuzzy` or `sklearn`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor,
)

In [10]:
#  feature creation ex - spilt dates
Date_Agent = LlmAgent(
    name="DateAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to Create Features by splitting Date/Time columns in the variable `df`.

    **CRITICAL:** Load 'df.pkl'. Do NOT create dummy data. Extract features from 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Identify Candidates:**
       - Iterate through all columns.
       - Target columns where:
         - Dtype is already `datetime`.
         - OR Name contains: "date", "time", "joined", "created", "at" (case-insensitive) AND Dtype is `object`.
    
    2. **Safe Conversion:**
       - For candidates, attempt: `temp = pd.to_datetime(df[col], errors='coerce')`
       - **Validation:** Check the NaT (Null) rate of `temp`.
         - IF NaT rate > 50%: The column is likely NOT a real date. **SKIP IT.**
         - IF NaT rate <= 50%: Assign `df[col] = temp` and proceed to step 3.

    3. **Feature Splitting (The "Creation" Step):**
       - For every valid date column:
         - Create `{col}_year`: `df[col].dt.year`
         - Create `{col}_month`: `df[col].dt.month`
         - Create `{col}_day`: `df[col].dt.day`
         - Create `{col}_dow`: `df[col].dt.dayofweek` (0=Mon, 6=Sun)
         - Create `{col}_is_weekend`: `(df[col].dt.dayofweek >= 5).astype(int)`

    4. **Cleanup:**
       - DROP the original date column after extracting features (Models cannot digest raw timestamps).

    **OUTPUT RULES:**
    1. Output ONLY a Python code block.
    2. End with:
        1. `print(f"Date Features Created. New Shape: {df.shape}")` and `print(df.columns.tolist())`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Date features added. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [11]:
# remove duplicates data <1% for safety
duplicates_agent = LlmAgent(
    name="DuplicatesAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to remove duplicate data safely from a variable `df`.

    **CRITICAL:** Load 'df.pkl'. Remove duplicates from 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Calculate Duplicates:** Identify how many rows would be removed using exact row matching.
    2. **The 1% Safety Rule:** - Calculate the drop percentage: `(duplicates_count / total_rows) * 100`.
       - IF the drop percentage is **greater than 1%**: Do NOT modify `df`. Instead, `raise ValueError(f"Aborting: Duplicates exceed 1% safety limit. Found {pct}%")`.
       - IF the drop percentage is **less than or equal to 1%**: Remove the duplicates permanently from `df`.

    **OUTPUT RULES:**
    1. Your output MUST be ONLY a Python code block (```python ... ```).
    2. Do NOT write any text/explanation before or after the code block.
    3. If successful, end by:
        1. printing: `print(f"Successfully dropped {dropped_count} rows. New shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Duplicates removed. Saved to df.pkl")
    4. Use ONLY standard libraries and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [12]:
# groups rare categories into "other" <1%, 0.5% individually
grouper_agent = LlmAgent(
    name="GrouperAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to group rare categories into 'Other' for all string/object columns.

    **CRITICAL:** Load 'df.pkl'. Group rare categories in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Iterate:** Loop through every column in `df` where `dtype == 'object'`.
    2. **Identify Candidates:** Inside the loop, find categories that appear in **less than 0.5%** of the rows. These are "rare candidates".
    3. **The 1% Safety Rule:**
       - Calculate the **Total Impact**: Sum the counts of all "rare candidates" in that column.
       - Calculate the **Impact Percentage**: `(Total Impact / Total Rows) * 100`.
       - **IF Impact Percentage <= 1%**: Replace those rare categories with the string "Other".
       - **IF Impact Percentage > 1%**: Do NOT modify that column. Print a warning that grouping was skipped for safety.

    **OUTPUT RULES:**
    1. Your output MUST be ONLY a Python code block (```python ... ```).
    2. Do NOT write any text/explanation before or after the code block.
    3. The code MUST end with:
        1. A summary print for each column: `print(f"Column '{col}': Grouped {count} rows ({pct}%)")`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Grouping complete. Saved to df.pkl")
    4. Use ONLY standard libraries and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [13]:
# Drop rows if the missing data is minimal (<5%), columns if the feature is mostly empty (>50%)
# Use median to fill in if possible
# total thresholding to prevent cascading data loss
Null_agent = LlmAgent(
    name="NullAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to handle missing data (nulls).

    **CRITICAL:** Load 'df.pkl'. Handle nulls in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC - EXECUTE IN THIS ORDER:**
    1. **Column Cleanup:**
       - IF a column is missing > 50% data: Drop the **Column**.

    2. **Global Row Safety Check:**
       - Identify ALL rows that contain nulls in the remaining columns.
       - Calculate `total_rows_with_nulls`.
       - Calculate `loss_percentage = (total_rows_with_nulls / total_rows) * 100`.

    3. **Decision Branch:**
       - **IF loss_percentage < 5%:**
         - Drop ALL rows containing nulls. (Safe to drop).
       - **ELSE (If loss > 5%):**
         - Do NOT drop rows. Instead, Impute (Fill) data:
         - Numerics -> Fill with Median.
         - Object/String -> Fill with Mode (Top value).

    **OUTPUT RULES:**
    1. Output ONLY a Python code block.
    2. End with: 
        1. `print(f"Action taken: {'Dropped Rows' if loss < 5 else 'Imputed Data'}. New Shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Redundancy removed. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [14]:
# PCA - correlation matrix, drops var/features with little relevance
Correlation_Agent = LlmAgent(
    name="CorrelationAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` and `numpy` to remove **Redundant Features** (Multicollinearity) based on a correlation matrix.

    **CRITICAL:** Load 'df.pkl'. Drop correlated cols in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Preprocessing:**
       - Select ONLY numeric columns for calculation.
       - If fewer than 2 numeric columns exist, STOP and do nothing.

    2. **Calculate Correlation:**
       - Compute the absolute correlation matrix: `corr_matrix = df.select_dtypes(include=[np.number]).corr().abs()`

    3. **Identify Redundant Features (The "Irrelevant" ones):**
       - **Constraint:** We want to keep one variable and drop its duplicates.
       - Select the **Upper Triangle** of the correlation matrix (to avoid checking a column against itself or checking pairs twice).
       - Find columns where the correlation score is **> 0.95** (95% similar).
       - These columns provide "Little Relevance" (Information Gain) because they are duplicates of other columns.

    4. **Execution:**
       - Drop the identified redundant columns from the original `df`.

    **OUTPUT RULES:**
    1. Output ONLY a Python code block.
    2. You MUST import numpy as np.
    3. End with:
        1.`print(f"Dropped {len(to_drop)} redundant features: {to_drop}. New Shape: {df.shape}")`
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Redundancy removed. Saved to df.pkl")
    4. Use ONLY standard libraries, `numpy`, and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [15]:
# for numeric values, if skew() > 1, log transform for outliers (log(x+1)) if no neg values
# exclude ID rows and dates
Skew_Agent = LlmAgent(
    name="SkewAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` and `numpy` to fix positive skewness in the variable `df`.

    **CRITICAL:** Load 'df.pkl'. Fix skew in 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Filter:** Iterate ONLY through **numeric** columns (float/int).
    2. **ID Exclusion (CRITICAL):**
       - Check the column name.
       - **ID Checks:** IF the name contains "id", "ID", or "Id" (case-insensitive check): **SKIP** this column entirely. Do not transform identifiers.
       - **Date Part Checks:** IF name ends with "_year", "_month", "_day", "_dow", "_weekend" -> SKIP. (Do not skew-transform time features).
    3. **Check Constraints:** For the remaining numeric columns:
       - Check if the column contains **Any Negative Values**. If yes -> SKIP (Log is undefined for negatives).
       - Calculate the **Skewness** using `.skew()`.
    4. **Apply Transformation:**
       - **IF skew > 1** (High Positive Skew):
         - Apply `np.log1p(x)` to the entire column. (avoids log(0))

    **OUTPUT RULES:**
    1. Your output MUST be ONLY a Python code block (```python ... ```).
    2. You MUST import numpy as np.
    3. The code MUST end with:
        1. A summary loop printing: `print(f"Column '{col}': Skew from {old_skew} -> {new_skew}")` for changed columns only.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Skew fixed. Saved to df.pkl")
    4. Use ONLY standard libraries, `numpy`, and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [16]:
# one hot encoding for low cardinality
# label encoding for high and ordinal data
Encoding_Agent = LlmAgent(
    name="EncodingAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `pandas` to encode categorical data (One-Hot vs Label Encoding) in the variable `df`.

    **CRITICAL:** Load 'df.pkl'. Encode 'df.pkl', overwrite 'df.pkl'.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC & SAFETY CHECKS:**
    1. **Filter:** Iterate through columns where `dtype == 'object'` (Categorical).
    2. **ID Exclusion:** IF column name contains "id", "ID", or "Id" -> SKIP.
    3. **Cardinality Check (Threshold = 10):**
       - Calculate `unique_count = df[col].nunique()`
    4. **Apply Encoding:**
       - **CASE A: Low Cardinality (unique_count < 10):**
         - Apply **One-Hot Encoding** using `pd.get_dummies`.
         - Ensure `prefix=col_name` is used to track origin.
         - **CRITICAL:** Concatenate the new columns to `df` and DROP the original string column.
       - **CASE B: High Cardinality (unique_count >= 10):**
         - Apply **Label Encoding**.
         - Implementation: Convert to category type and use code accessor: `df[col] = df[col].astype('category').cat.codes`
         - This preserves the column as a single numerical feature.

    **OUTPUT RULES:**
    1. Output ONLY a Python code block.
    2. The code MUST end with:
        1. `print(f"Encoding Complete. New Shape: {df.shape}")` and `print(df.dtypes)`.
        2. df.to_pickle('/kaggle/working/df.pkl'); print("Encoding complete. Saved to df.pkl")
    3. Use ONLY standard libraries and `pandas`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [17]:
# feature engineering - z score standardization for reducing scale
# ^ spilt data train and test, prevents leakage, then transform test set using precalculated values (mean/std)
# Not using normalization here since it's weaker to outliers and is better on Neural Nets/KNNs/Images
Scaler_Agent = LlmAgent(
    name="ScalerAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `sklearn` to split the data and perform Z-Score Standardization while preventing Data Leakage.

    **CRITICAL:** Load 'df.pkl'. DO NOT create dummy data. Save 'train_df.pkl' and 'test_df.pkl' to the current directory.

    **INPUT CONTEXT:**
    - load df by: df = pd.read_pickle('/kaggle/working/df.pkl')

    **LOGIC - EXECUTE IN THIS ORDER:**
    1. **Split Data (Prevent Leakage):**
       - Use `train_test_split` to create `train_df` and `test_df`: train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
       - **CRITICAL:** Do this *before* any scaling.

    2. **Identify Features:**
       - Identify **Numeric** columns in `train_df`.
       - **Exclusion:** Do NOT scale columns that appear to be ID keys or the Target variable (if recognizable). If unknown, scale all numerics.

    3. **Apply Z-Score Standardization:**
       - Initialize `StandardScaler`.
       - **Step A (Train):** `.fit_transform()` the scaler on the **Numeric Columns of `train_df`**.
       - **Step B (Test):** `.transform()` the **Numeric Columns of `test_df`** using the scaler fitted on Train.
       - *Note:* This ensures the Test set is scaled using the Train set's Mean and Std Dev.

    **OUTPUT RULES:**
    1. Output ONLY a Python code block.
    2. You MUST import `train_test_split` and `StandardScaler`.
    3. **CRITICAL:** Use absolute paths for saving:
        - train_df.to_pickle('/kaggle/working/train_df.pkl')
        - test_df.to_pickle('/kaggle/working/test_df.pkl')
        - print("Split & Scaled. Created train_df.pkl and test_df.pkl")
    4. End with:
       - `print(f"Split Complete. Train Shape: {train_df.shape}, Test Shape: {test_df.shape}")`
       - `print("Z-Score Standardization applied safely.")`
    5. Ensure `train_df` and `test_df` are available variables.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [18]:
# Fixes Statistical Bias and Class Imbalance on Train set
# To prevent overfitting, we introduce jittering/noise injection
# ^ but obviously exclude target so the model doesn't predict that you have 1.2 siblings or smth
AutoBalance_Agent = LlmAgent(
    name="AutoBalanceAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a Python Data Engineering Expert. You DO NOT speak natural language. You ONLY output executable Python code.

    **YOUR GOAL:**
    Write a Python script using `sklearn`, `numpy`, and `pandas` to Balance Training Data safely using **Noise Injection** (Jittering).

    **CRITICAL:** Load 'train_df.pkl'. DO NOT create dummy data. Balance 'train_df.pkl' and overwrite 'train_df.pkl'.

    **INPUT CONTEXT:**
    - load `train_df` by: train_df = pd.read_pickle('/kaggle/working/train_df.pkl')

    **LOGIC - EXECUTE IN THIS ORDER:**
    1. **Auto-Detect Target (Generic Only):**
       - **Heuristic 1:** Check column names for: ["target", "label", "class", "outcome", "y"] (case-insensitive). Use the first match.
       - **Heuristic 2:** If no match, assume the **Last Column** is the target.
       - Save name as `target_col`.

    2. **Safety Check (Classification vs Regression):**
       - IF `train_df[target_col].nunique() > 20`: STOP. Do not balance regression data (high cardinality).

    3. **Check Imbalance:**
       - Calculate Ratio: `(Minority Count / Majority Count)`.
       - **IF Ratio >= 0.8(Already Balanced):** - Print "Data is balanced. No action taken."

    4. **ELSE IF Ratio < 0.8 (Imbalanced):**
       - **Step A:** Separate Majority and Minority dataframes.
       - **Step B:** Resample Minority to match Majority count (create copies).
       - **Step C:** Identify the *newly created copies*.
       - **Step D (Inject Noise):**
         - For the **Numerical Columns** (columns where dtype is numeric) of the copies ONLY:
         - **CRITICAL EXCLUSION:** You MUST exclude `target_col` from the list of columns to jitter. Never alter the target label.
         - For the remaining numerical feature columns of the copies:
             - Add random noise: `value = value + np.random.normal(0, 0.01 * std_dev)`.
         - *Note:* This prevents exact duplicates by shifting points by 1% of their standard deviation.
       - **Step E:** Concatenate Majority + Jittered Minority.

    **OUTPUT RULES:**
    1. Output ONLY a Python code block. Import `resample` from `sklearn.utils` and `numpy as np`.
    2. **CRITICAL:** Save to absolute path:
       - train_df.to_pickle('/kaggle/working/train_df.pkl')
    3. End with: `print(f"Target: '{target_col}'. Balanced with Jitter. New Shape: {train_df.shape}")`.

    Failure to follow these rules will result in a system error.
    """,
    code_executor=local_executor, 
)

In [19]:
# Overall Model Orchestrator
Mastermind = LlmAgent(
    name="Mastermind_agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    static_instruction="""
    You are a ruthlessly efficient Data Prepping Orchestrator.
    
    **YOUR MISSION:**
    Execute the following data engineering pipeline strictly in order. 
    You do not need to write code yourself; use the provided Tools to generate and execute the code.
    All agents must read/write from the CURRENT DIRECTORY (/kaggle/working).
    The tools will handle file I/O automatically using fixed filenames: 'df.pkl', 'train_df.pkl', 'test_df.pkl'.
    
    **PIPELINE SEQUENCE:**
    1.  **standardize_agent**: Clean formatting (df -> df).
    2.  **Date_Agent**: Extract time features (df -> df).
    3.  **duplicates_agent**: Remove rows (df -> df).
    4.  **grouper_agent**: Group rare categories (df -> df).
    5.  **Null_agent**: Impute or drop missing data (df -> df).
    6.  **Correlation_Agent**: Drop redundant features (df -> df).
    7.  **Skew_Agent**: Fix numeric skew (df -> df).
    8.  **Encoding_Agent**: Categorical to Numerical (df -> df).
    9.  **Scaler_Agent**: SPLIT into Train/Test and Scale (df -> train_df, test_df) (Saves train_df.pkl and test_df.pkl).
    10. **AutoBalance_Agent**: Balance the TRAINING set only (train_df -> train_df)(Overwrites train_df.pkl). 
        *DO NOT touch testdf in this step.*

    **OUTPUT RULES:**
    1. Do not output the actual dataframes as text (they are too large).
    2. Once Step 10 is finished, output a final confirmation: 
       "✅ Pipeline Complete. Variables 'train_df' and 'test_df' are ready for modeling."

    Failure to follow the sequence will result in immediate termination.
    """,
    tools=[
        AgentTool(agent=standardize_agent),
        AgentTool(agent=Date_Agent),
        AgentTool(agent=duplicates_agent),
        AgentTool(agent=grouper_agent),
        AgentTool(agent=Null_agent),
        AgentTool(agent=Correlation_Agent),
        AgentTool(agent=Skew_Agent),
        AgentTool(agent=Encoding_Agent),
        AgentTool(agent=Scaler_Agent),
        AgentTool(agent=AutoBalance_Agent),
    ],
)

print("✅ The Bane of Interns is online.🤖")

✅ The Bane of Interns is online.🤖


In [20]:
# Save a copy for comparing later
df_raw_snapshot = df.copy()
runner = InMemoryRunner(agent=Mastermind)
await runner.run_debug("Start the pipeline. df is at '/kaggle/working/df.pkl'")


 ### Created new session: debug_session_id

User > Start the pipeline. df is at '/kaggle/working/df.pkl'


<string>:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
<string>:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
<string>:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
<string>:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/p

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250, model: gemini-2.5-flash\nPlease retry in 6.947087797s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.

In [ ]:
# Time to compare

def print_comparison_report(raw, train, test):
    print("\n" + "="*40)
    print("🧪 PIPELINE VALIDATION REPORT")
    print("="*40)

    # 1. SHAPE & DROPS
    total_final = len(train) + len(test)
    dropped = len(raw) - total_final
    print(f"\n1. DATA VOLUME:")
    print(f"   - Original: {len(raw)} rows")
    print(f"   - Final:    {total_final} rows")
    print(f"   - Dropped:  {dropped} rows ({(dropped/len(raw)):.1%} loss)")

    # 2. NULLS
    print(f"\n2. NULL CHECK:")
    print(f"   - Remaining Nulls: {train.isnull().sum().sum()} (Must be 0)")

    # 3. COLUMNS (Encoding check)
    print(f"\n3. DIMENSIONS:")
    print(f"   - Orig Cols:  {raw.shape[1]}")
    print(f"   - Final Cols: {train.shape[1]}")
    
    # 4. SKEW (Log Transform Check)
    if 'Transaction_Amt' in raw.columns and 'Transaction_Amt' in train.columns:
        print(f"\n4. SKEW CORRECTION:")
        print(f"   - Orig Max:  ${raw['Transaction_Amt'].max():,.2f}")
        print(f"   - Final Max: {train['Transaction_Amt'].max():.4f} (Scaled)")

    # 5. BALANCE CHECK
    target_col = 'Target_Label'
    if target_col in train.columns:
        print(f"\n5. CLASS BALANCE ({target_col}):")
        tc = train[target_col].value_counts(normalize=True)
        print(f"   - Train (Balanced): 0: {tc.get(0,0):.2f} | 1: {tc.get(1,0):.2f}")
        
        testc = test[target_col].value_counts(normalize=True)
        print(f"   - Test (Natural):   0: {testc.get(0,0):.2f} | 1: {testc.get(1,0):.2f}")

train_abs_path = '/kaggle/working/train_df.pkl'
test_abs_path = '/kaggle/working/test_df.pkl'

if os.path.exists(train_abs_path):
    print(f"\n✅ SUCCESS: Found {train_abs_path}")
    final_train = pd.read_pickle(train_abs_path)
    final_test = pd.read_pickle(test_abs_path)
    print_comparison_report(df_raw_snapshot, final_train, final_test)
else:
    print(f"\n❌ FAILURE: File still missing at {train_abs_path}")

In [ ]:
print("📍 Current Working Directory: /kaggle/working")
print("-" * 30)

print("🔎 Scanning entire file system for '.pkl' files...")
found_files = []

# Walk through the directory tree
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".pkl"):
            full_path = os.path.join(root, file)
            print(f"   found: {full_path}")
            found_files.append(full_path)

print("-" * 30)
if not found_files:
    print("❌ No .pkl files found. The Agent's file system might have been wiped.")
else:
    print(f"✅ Found {len(found_files)} files. Copy the path above to load them.")

In [ ]:
import pandas as pd

# Load the file form disk to be sure we check the result
df_processed = pd.read_pickle('/kaggle/working/df.pkl') 

# 1. Check if Shape Changed (Rows dropped?)
shape_changed = df_processed.shape != df_raw_snapshot.shape

# 2. Check if Values Changed (Scaling/Encoding?)
# .equals() returns True if they are identical, so we invert it with 'not'
content_changed = not df_processed.equals(df_raw_snapshot)

print(f"Rows/Cols Changed: {shape_changed}  (Raw: {df_raw_snapshot.shape} -> New: {df_processed.shape})")
print(f"Values Changed:    {content_changed}")

if not content_changed:
    print("\n❌ FALSE: Data is identical to raw input. Agents failed to save changes.")
else:
    print("\n✅ TRUE: Data has been transformed.")

In [ ]:
# I am just a recent CS grad, so the solutions provided here are a far cry from perfect,
# but if there are any questions, please contact me at https://www.linkedin.com/in/yulin-lin-0a05201ab/.